# Objective

Generate features for machine learning algorithms. The goal is to use unsupervised machine learning algorithms to cluster similar instances together, e.g., highly volatile instances, and/or cheaper least volatile instances.

This will help us in the migration between spot instances, i.e., if we need to move from one spot instance that is going to expire, which should be the next of where we should go next.

# Code

## Load libs

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import random
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from src.data.helpers import load_aws_dataset
from src.data.helpers import remove_consecutive_repeated_price_entries
from src.data.helpers import calc_pdf_price_update_interval_seconds
from src.data.helpers import generate_price_update_interval

## Input params

In [15]:
data_compression = 'zip'

interim_data_dir = '../data/interim'
interim_data_filename = 'aws_filtered_sample.csv.zip'

processed_data_dir = '../data/processed'
processed_data_filename = 'step_5_features_pricing_var.csv.zip'

## Load data

In [3]:
file = f'{interim_data_dir}/{interim_data_filename}'
data = load_aws_dataset(file)
print(data.shape)
data.head()

(1666418, 4)


/Users/walterwong/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Timestamp,AvailabilityZone,InstanceType,SpotPrice
0,2020-06-01 00:00:04,us-east-1f,r5d.large,0.0356
1,2020-06-01 00:00:04,us-east-1c,r5d.large,0.0356
2,2020-06-01 00:00:04,us-east-1d,r5d.large,0.0356
3,2020-06-01 00:00:04,us-east-1b,r5d.large,0.0356
4,2020-06-01 00:00:50,us-west-2c,r5.2xlarge,0.1560


### Pivot table to wide format

Wide format allows to have a price column for each instance

In [4]:
%%time

df = data.query('AvailabilityZone == "us-east-1a"')\
         .drop('AvailabilityZone', axis=1)

print(df.shape)

# Pivot table to change a wide format for the data. Thus, we can remove
# instances that do not have any price update.
# Dropping MultiIndex column 'SpotPrice' as there is no use for it.
pvt = df.pivot_table(index=['Timestamp'], 
                     columns=['InstanceType'])\
        .droplevel(0, axis=1)

pvt.head()

(58756, 3)
CPU times: user 381 ms, sys: 185 ms, total: 565 ms
Wall time: 563 ms


InstanceType,a1.2xlarge,a1.4xlarge,a1.large,a1.medium,a1.metal,a1.xlarge,c1.medium,c1.xlarge,c3.2xlarge,c3.4xlarge,...,x1e.4xlarge,x1e.8xlarge,x1e.xlarge,z1d.12xlarge,z1d.2xlarge,z1d.3xlarge,z1d.6xlarge,z1d.large,z1d.metal,z1d.xlarge
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 00:05:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:05:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:06:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:13:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:14:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Feature Engineering

As features, we will extract the following:
1. Price variation function: extract the probabilistic density function (pdf) of price variation;
2. Volatility curve: this represents the  pdf of the number of price changes for a given instance;
3. Price update interval curve: represents the pdf of price update interval

We will use these three pdfs to cluster our instances together.

### Price variation (PDF)

In [5]:
var_list = [{'Instance': cname, 'price_var': cdf.dropna().var()} for cname, cdf in pvt.items()]
var_df = pd.DataFrame(var_list).set_index('Instance').T
var_df

Instance,a1.2xlarge,a1.4xlarge,a1.large,a1.medium,a1.metal,a1.xlarge,c1.medium,c1.xlarge,c3.2xlarge,c3.4xlarge,...,x1e.4xlarge,x1e.8xlarge,x1e.xlarge,z1d.12xlarge,z1d.2xlarge,z1d.3xlarge,z1d.6xlarge,z1d.large,z1d.metal,z1d.xlarge
price_var,7.960510e-34,3.187746e-33,4.884844e-07,3.109574e-36,7.944461e-34,4.970142e-35,1.489900e-34,0.002035,0.000009,0.000193,...,2.441052e-30,9.763170e-30,1.525337e-31,2.787220e-07,0.0008,0.000007,0.000098,0.000588,4.981209e-30,0.000191


### Volatility (PDF)



In [6]:
# Iterate over each instance type, drop NaN and get the price distribution.
volatility_list = [cdf.dropna().describe(include='all').to_frame() for _, cdf in pvt.items()]
volatility_pdf = pd.concat(volatility_list, axis=1).round(3)
volatility_pdf

,a1.2xlarge,a1.4xlarge,a1.large,a1.medium,a1.metal,a1.xlarge,c1.medium,c1.xlarge,c3.2xlarge,c3.4xlarge,...,x1e.4xlarge,x1e.8xlarge,x1e.xlarge,z1d.12xlarge,z1d.2xlarge,z1d.3xlarge,z1d.6xlarge,z1d.large,z1d.metal,z1d.xlarge
count,31.000,30.000,63.000,31.000,33.000,32.000,97.000,357.000,111.000,114.000,...,97.000,98.000,99.00,104.000,296.000,112.000,119.000,182.000,98.000,243.000
mean,0.067,0.134,0.022,0.008,0.134,0.034,0.013,0.140,0.142,0.298,...,1.001,2.002,0.25,1.339,0.248,0.336,0.673,0.077,1.339,0.129
std,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.045,0.003,0.014,...,0.000,0.000,0.00,0.001,0.028,0.003,0.010,0.024,0.000,0.014
min,0.067,0.134,0.020,0.008,0.134,0.034,0.013,0.074,0.137,0.272,...,1.001,2.002,0.25,1.339,0.223,0.335,0.670,0.056,1.339,0.112
25%,0.067,0.134,0.021,0.008,0.134,0.034,0.013,0.110,0.141,0.284,...,1.001,2.002,0.25,1.339,0.228,0.335,0.670,0.056,1.339,0.119
50%,0.067,0.134,0.022,0.008,0.134,0.034,0.013,0.130,0.142,0.300,...,1.001,2.002,0.25,1.339,0.234,0.335,0.670,0.068,1.339,0.123
75%,0.067,0.134,0.022,0.008,0.134,0.034,0.013,0.154,0.143,0.310,...,1.001,2.002,0.25,1.339,0.276,0.335,0.670,0.096,1.339,0.143
max,0.067,0.134,0.023,0.008,0.134,0.034,0.013,0.289,0.149,0.318,...,1.001,2.002,0.25,1.344,0.317,0.348,0.707,0.140,1.339,0.160


### Merge Price and Volatility PDFs

In [7]:
res_df = pd.concat([volatility_pdf, var_df], axis=0).round(3)
res_df

,a1.2xlarge,a1.4xlarge,a1.large,a1.medium,a1.metal,a1.xlarge,c1.medium,c1.xlarge,c3.2xlarge,c3.4xlarge,...,x1e.4xlarge,x1e.8xlarge,x1e.xlarge,z1d.12xlarge,z1d.2xlarge,z1d.3xlarge,z1d.6xlarge,z1d.large,z1d.metal,z1d.xlarge
count,31.000,30.000,63.000,31.000,33.000,32.000,97.000,357.000,111.000,114.000,...,97.000,98.000,99.00,104.000,296.000,112.000,119.000,182.000,98.000,243.000
mean,0.067,0.134,0.022,0.008,0.134,0.034,0.013,0.140,0.142,0.298,...,1.001,2.002,0.25,1.339,0.248,0.336,0.673,0.077,1.339,0.129
std,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.045,0.003,0.014,...,0.000,0.000,0.00,0.001,0.028,0.003,0.010,0.024,0.000,0.014
min,0.067,0.134,0.020,0.008,0.134,0.034,0.013,0.074,0.137,0.272,...,1.001,2.002,0.25,1.339,0.223,0.335,0.670,0.056,1.339,0.112
25%,0.067,0.134,0.021,0.008,0.134,0.034,0.013,0.110,0.141,0.284,...,1.001,2.002,0.25,1.339,0.228,0.335,0.670,0.056,1.339,0.119
50%,0.067,0.134,0.022,0.008,0.134,0.034,0.013,0.130,0.142,0.300,...,1.001,2.002,0.25,1.339,0.234,0.335,0.670,0.068,1.339,0.123
75%,0.067,0.134,0.022,0.008,0.134,0.034,0.013,0.154,0.143,0.310,...,1.001,2.002,0.25,1.339,0.276,0.335,0.670,0.096,1.339,0.143
max,0.067,0.134,0.023,0.008,0.134,0.034,0.013,0.289,0.149,0.318,...,1.001,2.002,0.25,1.344,0.317,0.348,0.707,0.140,1.339,0.160
price_var,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,...,0.000,0.000,0.00,0.000,0.001,0.000,0.000,0.001,0.000,0.000


### Export to csv

In [16]:
res_df.to_csv(f'{processed_data_dir}/{processed_data_filename}', compression=data_compression)

### Price Update Interval (PDF)

In [8]:
pvt.head()

InstanceType,a1.2xlarge,a1.4xlarge,a1.large,a1.medium,a1.metal,a1.xlarge,c1.medium,c1.xlarge,c3.2xlarge,c3.4xlarge,...,x1e.4xlarge,x1e.8xlarge,x1e.xlarge,z1d.12xlarge,z1d.2xlarge,z1d.3xlarge,z1d.6xlarge,z1d.large,z1d.metal,z1d.xlarge
Timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-01 00:05:28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:05:31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:06:06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:13:29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-01 00:14:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Remove repeated price entries without actual change

sample_instance = pvt.loc[:, 'r5n.16xlarge']
sample_instance_norepeat = remove_consecutive_repeated_price_entries(sample_instance)
sample_instance_norepeat

Timestamp
2020-06-01 03:51:56    1.1501
2020-06-01 08:54:20    1.1504
2020-06-01 15:36:25    1.1502
2020-06-01 22:44:34    1.1519
2020-06-02 04:36:46    1.1546
                        ...  
2020-08-30 19:28:26    1.4237
2020-08-31 00:15:40    1.4264
2020-08-31 05:27:33    1.4300
2020-08-31 10:30:13    1.4345
2020-08-31 22:10:12    1.4337
Name: r5n.16xlarge, Length: 362, dtype: float64

In [10]:
update_interval_dist = calc_pdf_price_update_interval_seconds(sample_instance_norepeat)
update_interval_dist

,r5n.16xlarge
count,361.000000
mean,21962.038781
std,3465.494613
min,2051.000000
25%,19167.000000
50%,22208.000000
75%,24271.000000
max,41999.000000


In [11]:
uplist = []
for cname, cdf in pvt.items():
    uplist.append((cdf.pipe(remove_consecutive_repeated_price_entries)\
         .pipe(calc_pdf_price_update_interval_seconds)))

    
res_df = pd.concat(uplist, axis=1).iloc[1:].dropna(axis=1, how='all')
res_df

,a1.large,c1.xlarge,c3.2xlarge,c3.4xlarge,c3.8xlarge,c3.large,c3.xlarge,c4.2xlarge,c4.4xlarge,c4.8xlarge,...,t3a.micro,t3a.nano,t3a.small,t3a.xlarge,z1d.12xlarge,z1d.2xlarge,z1d.3xlarge,z1d.6xlarge,z1d.large,z1d.xlarge
mean,48002.215686,23118.774854,23718.355769,23279.242991,22547.162162,396009.000000,109994.909091,23105.559767,22788.354467,22270.766197,...,2.524606e+05,2.023946e+05,138695.754386,58808.932836,6.098452e+05,2.974318e+04,1.475865e+05,2.859488e+05,7.163979e+04,3.906406e+04
std,34996.722451,5713.610456,6900.502253,6876.125450,5310.423968,209784.830316,122003.650071,7306.742782,5815.030315,4706.520829,...,4.010504e+05,3.386562e+05,154888.722623,54426.187035,1.009967e+06,1.069084e+05,5.115205e+05,9.471590e+05,2.927462e+05,1.539963e+05
min,17637.000000,11821.000000,16564.000000,16493.000000,16623.000000,208548.000000,19172.000000,16429.000000,481.000000,4453.000000,...,1.665700e+04,3.631000e+03,16602.000000,7105.000000,7.377000e+03,7.120000e+03,1.016000e+03,8.784000e+03,1.660500e+04,3.661000e+03
25%,24173.000000,19163.000000,19655.000000,19383.500000,19635.500000,268342.500000,42589.500000,19141.500000,19650.500000,19168.000000,...,4.732500e+04,2.285550e+04,43341.000000,24529.500000,1.622550e+04,1.931575e+04,1.767000e+04,2.000375e+04,1.966525e+04,1.921100e+04
50%,40816.000000,22421.000000,23159.500000,22198.000000,21681.000000,343734.000000,77583.000000,21712.000000,22661.000000,21727.000000,...,1.235480e+05,2.958600e+04,94720.000000,42921.000000,1.605160e+05,2.221250e+04,1.961000e+04,2.294150e+04,2.266200e+04,2.266600e+04
75%,57136.500000,25161.750000,25578.500000,24936.500000,24207.500000,471400.500000,102504.250000,24979.000000,24708.000000,24681.500000,...,3.042580e+05,2.486692e+05,176723.000000,70133.500000,7.541358e+05,2.517075e+04,2.519900e+04,2.517775e+04,2.569675e+04,2.573400e+04
max,207049.000000,45531.000000,54365.000000,58437.000000,46775.000000,688020.000000,522133.000000,73019.000000,78592.000000,50882.000000,...,2.103855e+06,1.393036e+06,925199.000000,374915.000000,2.110972e+06,1.762407e+06,2.131480e+06,4.236366e+06,2.734244e+06,2.100592e+06


In [12]:


res_df = generate_price_update_interval(pvt)
res_df

,a1.large,c1.xlarge,c3.2xlarge,c3.4xlarge,c3.8xlarge,c3.large,c3.xlarge,c4.2xlarge,c4.4xlarge,c4.8xlarge,...,t3a.micro,t3a.nano,t3a.small,t3a.xlarge,z1d.12xlarge,z1d.2xlarge,z1d.3xlarge,z1d.6xlarge,z1d.large,z1d.xlarge
mean,48002.215686,23118.774854,23718.355769,23279.242991,22547.162162,396009.000000,109994.909091,23105.559767,22788.354467,22270.766197,...,2.524606e+05,2.023946e+05,138695.754386,58808.932836,6.098452e+05,2.974318e+04,1.475865e+05,2.859488e+05,7.163979e+04,3.906406e+04
std,34996.722451,5713.610456,6900.502253,6876.125450,5310.423968,209784.830316,122003.650071,7306.742782,5815.030315,4706.520829,...,4.010504e+05,3.386562e+05,154888.722623,54426.187035,1.009967e+06,1.069084e+05,5.115205e+05,9.471590e+05,2.927462e+05,1.539963e+05
min,17637.000000,11821.000000,16564.000000,16493.000000,16623.000000,208548.000000,19172.000000,16429.000000,481.000000,4453.000000,...,1.665700e+04,3.631000e+03,16602.000000,7105.000000,7.377000e+03,7.120000e+03,1.016000e+03,8.784000e+03,1.660500e+04,3.661000e+03
25%,24173.000000,19163.000000,19655.000000,19383.500000,19635.500000,268342.500000,42589.500000,19141.500000,19650.500000,19168.000000,...,4.732500e+04,2.285550e+04,43341.000000,24529.500000,1.622550e+04,1.931575e+04,1.767000e+04,2.000375e+04,1.966525e+04,1.921100e+04
50%,40816.000000,22421.000000,23159.500000,22198.000000,21681.000000,343734.000000,77583.000000,21712.000000,22661.000000,21727.000000,...,1.235480e+05,2.958600e+04,94720.000000,42921.000000,1.605160e+05,2.221250e+04,1.961000e+04,2.294150e+04,2.266200e+04,2.266600e+04
75%,57136.500000,25161.750000,25578.500000,24936.500000,24207.500000,471400.500000,102504.250000,24979.000000,24708.000000,24681.500000,...,3.042580e+05,2.486692e+05,176723.000000,70133.500000,7.541358e+05,2.517075e+04,2.519900e+04,2.517775e+04,2.569675e+04,2.573400e+04
max,207049.000000,45531.000000,54365.000000,58437.000000,46775.000000,688020.000000,522133.000000,73019.000000,78592.000000,50882.000000,...,2.103855e+06,1.393036e+06,925199.000000,374915.000000,2.110972e+06,1.762407e+06,2.131480e+06,4.236366e+06,2.734244e+06,2.100592e+06
